In [1]:
import os
from pathlib import Path

root_directory = Path("C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main")
os.chdir(root_directory)
print(root_directory)

## Imports


In [8]:
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim, Patchcore, Fastflow

In [9]:
datamodule = MVTec(num_workers=0)
datamodule = MVTec(category="bottle")
datamodule.prepare_data()  
datamodule.setup()  

i, data = next(enumerate(datamodule.val_dataloader()))
print(data.keys())

dict_keys(['image_path', 'label', 'image', 'mask'])


Let's check the shapes of the input images and masks.


In [10]:
model = Fastflow(backbone="resnet18", flow_steps=8)
datamodule2 = MVTec(num_workers=5)

In [11]:
Fastflow??

Init signature:
Fastflow(
    backbone: str = 'resnet18',
    pre_trained: bool = True,
    flow_steps: int = 8,
    conv3x3_only: bool = False,
    hidden_ratio: float = 1.0,
) -> None
Source:        
class Fastflow(AnomalyModule):
    """PL Lightning Module for the FastFlow algorithm.

    Args:
        backbone (str): Backbone CNN network
            Defaults to ``resnet18``.
        pre_trained (bool, optional): Boolean to check whether to use a pre_trained backbone.
            Defaults to ``True``.
        flow_steps (int, optional): Flow steps.
            Defaults to ``8``.
        conv3x3_only (bool, optinoal): Use only conv3x3 in fast_flow model.
            Defaults to ``False``.
        hidden_ratio (float, optional): Ratio to calculate hidden var channels.
            Defaults to ``1.0`.
    """

    def __init__(
        self,
        backbone: str = "resnet18",
        pre_trained: bool = True,
        flow_steps: int = 8,
        conv3x3_only: bool = False,
        hidd

In [12]:
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=5,
    ),
]

In [13]:
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto",  
    devices=1,
    logger=False,
)
engine.fit(model=model, datamodule=datamodule2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
No implementation of `configure_transforms` was provided in the Lightning model. Using default transforms from the base class. This may not be suitable for your use case. Please override `configure_transforms` in your model.
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\loops\utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | loss                  | FastflowLoss             | 0     
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax        

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\core\module.py:494: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Validation


In [14]:
test_results = engine.test(
    model=model,
    datamodule=datamodule2,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Fastflow\MVTec\bottle\v3\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Fastflow\MVTec\bottle\v3\weights\lightning\model.ckpt
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9402597546577454     │
│       image_F1Score       │    0.9909583926200867     │
│        pixel_AUROC        │    0.47910240292549133    │
└───────────────────────────┴───────────────────────────┘

## OpenVINO Inference

Now that we trained and tested a model, we could check a single inference result using OpenVINO inferencer object. This will demonstrate how a trained model could be used for inference.


Before we can use OpenVINO inference, let's export the model to OpenVINO format first.


In [15]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:1209: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp

WindowsPath('C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/results/Fastflow/MVTec/bottle/latest/weights/openvino/model.xml')

### Load the OpenVINO Model

By default, the output files are saved into `results` directory. Let's check where the OpenVINO model is stored.


In [17]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Fastflow\MVTec\bottle\latest


In [18]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [19]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  
    metadata=metadata,  
    device="CPU",  
)

In [20]:
print(openvino_model_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Fastflow\MVTec\bottle\latest\weights\openvino\model.bin


In [21]:
import os

# Define la ruta al  que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/broken_large/"

# Vamos los archivos del directorio
files = os.listdir(directory)

# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")


Archivo: 000.png, Score: 0.7580711438470964, Label: 1
Archivo: 001.png, Score: 0.7649772202254439, Label: 1
Archivo: 002.png, Score: 0.7423525139623178, Label: 1
Archivo: 003.png, Score: 0.6604687551204866, Label: 1
Archivo: 004.png, Score: 0.5613591216109328, Label: 1
Archivo: 005.png, Score: 0.7178153063026824, Label: 1
Archivo: 006.png, Score: 0.6501098670292219, Label: 1
Archivo: 007.png, Score: 0.6614819645972769, Label: 1
Archivo: 008.png, Score: 0.8400554581468593, Label: 1
Archivo: 009.png, Score: 0.7657009251034187, Label: 1
Archivo: 010.png, Score: 0.6921073069869126, Label: 1
Archivo: 011.png, Score: 0.7682043483486563, Label: 1
Archivo: 012.png, Score: 0.7754180247787219, Label: 1
Archivo: 013.png, Score: 0.762975102174197, Label: 1
Archivo: 014.png, Score: 0.5984443368581569, Label: 1
Archivo: 015.png, Score: 0.7412079029614466, Label: 1
Archivo: 016.png, Score: 0.5844993331293387, Label: 1
Archivo: 017.png, Score: 0.7144964326479203, Label: 1
Archivo: 018.png, Score: 0.58

In [22]:
import os

# Define la ruta al directorio que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/good/"

# Vamos los archivos del directorio
files = os.listdir(directory)
# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")

Archivo: 000.png, Score: 0.5680719231518955, Label: 1
Archivo: 001.png, Score: 0.5817840639936747, Label: 1
Archivo: 002.png, Score: 0.522107570378799, Label: 1
Archivo: 003.png, Score: 0.4813354122645816, Label: 0
Archivo: 004.png, Score: 0.5487233771215362, Label: 1
Archivo: 005.png, Score: 0.7026880512191455, Label: 1
Archivo: 006.png, Score: 0.6282424520729247, Label: 1
